In [1]:
#%%
from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)

In [2]:
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
def value_counts_list(lst):
    """
    Computes the frequency count of unique elements in a list and returns a dictionary, sorted by frequency count in
    descending order.

    Args:
    - lst (list): List of elements

    Returns:
    - dict: Dictionary with unique elements as keys and their frequency count as values, sorted by frequency count
    in descending order
    """
    value_counts = {}
    for item in lst:
        if item in value_counts:
            value_counts[item] += 1
        else:
            value_counts[item] = 1
    sorted_value_counts = dict(sorted(value_counts.items(), key=lambda x: x[1], reverse=True))
    return sorted_value_counts

def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [4]:
original_data = pd.read_csv('data_aa/aa_rpoB.csv', header=None)
original_featrues = pd.read_csv('data_aa/RIF_MIC.csv', header=None)
data = original_data

target = original_featrues

train_data_index = np.random.choice(data.shape[0], size=int(data.shape[0]*0.8), replace=False)
all_indices = np.arange(data.shape[0])
test_data_index = np.setdiff1d(all_indices, train_data_index)

train_data = data.iloc[train_data_index,:]
train_target = target.iloc[train_data_index,:]
train_data = train_data.reset_index(drop=True)
train_target = train_target.reset_index(drop=True)
#don't touch test data, split out validation data from training data during training
test_data = data.iloc[test_data_index,:]
test_target = target.iloc[test_data_index,:]
test_data = test_data.reset_index(drop=True)
test_target = test_target.reset_index(drop=True)

In [5]:
class Dataset(torch.utils.data.Dataset): #? what's the difference between using inheritance and not?
    def __init__(
        self,
        train_df,
        mic_df,
        transform = None,
    ):
        self.transform = transform
        self.train_df = train_df
        self.mic_df = mic_df
        if not self.train_df.index.equals(self.mic_df.index):
            raise ValueError(
                "Indices of training data and resistance data don't match up"
            )

    def __getitem__(self, index):
        """
        numerical index --> get `index`-th sample
        string index --> get sample with name `index`
        """
        if isinstance(index, int):
            train = self.train_df.iloc[index]
            mic = self.mic_df.loc[index]
            
        elif isinstance(index, str):
            trains = self.train_df.loc[index]
            mic = self.mic_df.loc[index]
        else:
            raise ValueError(
                "Index needs to be an integer or a sample name present in the dataset"
            )
        
        if self.transform:
            self.mic_mean = self.mic_df.mean()
            self.mic_std = self.mic_df.std()
            mic = (mic - self.mic_mean) / self.mic_std
        
        return  torch.tensor(train),  torch.tensor(mic)
    def __len__(self):
        return self.mic_df.shape[0]
    
training_dataset = Dataset(train_data, train_target, transform=False)
train_dataset, val_dataset = random_split(training_dataset, [int(len(training_dataset)*0.8), len(training_dataset)-int(len(training_dataset)*0.8)])

In [6]:
torch.tensor(train_target.loc[1])

tensor([0.1200], dtype=torch.float64)

In [7]:
def get_masked_loss(loss_fn):
    """
    Returns a loss function that ignores NaN values
    """

    def masked_loss(y_true, y_pred):
        y_pred = y_pred.view(-1, 1)  # Ensure y_pred has the same shape as y_true and non_nan_mask
        # ic(y_true)
        non_nan_mask = ~y_true.isnan()
        # ic(non_nan_mask)
        y_true_non_nan = y_true[non_nan_mask]
        y_pred_non_nan = y_pred[non_nan_mask]

        return loss_fn(y_pred_non_nan, y_true_non_nan)

    return masked_loss

masked_MSE = get_masked_loss(torch.nn.MSELoss())

# Model

In [8]:
class Model(nn.Module):
    def __init__(self, in_channel = 869, first_h_layer = 469, out_channel=1, batch_size=1, dropout_rate=0.0, num_dense_layers=3, filter_scaling_factor=1.5):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.in_channel = in_channel
        self.first_h_layer = first_h_layer
        self.out_channel = out_channel
        self.dense_dropout_rate = dropout_rate
        self.num_dense_layers = num_dense_layers
        self.filter_scaling_factor=filter_scaling_factor
        
        self.dense_layers = nn.ModuleList()
        for i in range(self.num_dense_layers):
            layer = self._dense_layer(100,100)
            self.dense_layers.append(layer)
        
        # current_num_filters = self.cnn_output_size(self.in_channel, 12)
        current_num_filters = self.cnn_output_size_multilayer(self.in_channel, 12, num_layers=2)
        num_dense_neurons = 100
        num_dense_layers = 2
        self.dense_layers = nn.ModuleList(
            self._dense_layer(input_dim, num_dense_neurons)
            for input_dim in [current_num_filters]
            + [num_dense_neurons] * (num_dense_layers - 1) #how does this work?
        )
            # current_num_filters = int(current_num_filters * filter_scaling_factor)

        # self.feature_extraction = nn.Conv1d(in_channels, hidden, kernel_size=kernel_size),]
        # self.starting_layers = nn.Sequential(
        #     nn.Linear(self.in_channel, self.first_h_layer),
        #     nn.BatchNorm1d(self.first_h_layer),
        #     nn.ReLU(),
        #     nn.Dropout(self.dense_dropout_rate),  # Dropout layer after the first ReLU
        #     nn.Linear(self.first_h_layer, 100),
        #     nn.BatchNorm1d(100),
        #     nn.ReLU(),
        #     nn.Dropout(self.dense_dropout_rate))  # Dropout layer after the first ReLU
        
        self.starting_layers = nn.Sequential(
            nn.Conv1d(1, 1, 12),
            nn.BatchNorm1d(1),
            nn.ReLU(),
            nn.Dropout(self.dense_dropout_rate),  # Dropout layer after the first ReLU
            nn.Conv1d(1, 1, 12),
            nn.BatchNorm1d(1),
            nn.ReLU(),
            nn.Dropout(self.dense_dropout_rate))  # Dropout layer after the first ReLU
            
        self.out_layer = nn.Linear(100, self.out_channel)
        
        self.apply(self.init_weights)
        
    # def cnn_output_size(self, n, f, p = 0, s = 1):
    #     '''
    #     n: input size
    #     f: kernel size
    #     p: padding
    #     s: stride
    #     '''
    #     return int((n - f + 2 * p) / s) + 1
    
    def cnn_output_size_multilayer(self, n, f, p=0, s=1, num_layers=1):
        '''
        n: input size
        f: kernel size (assuming the same for all layers for simplicity)
        p: padding (assuming the same for all layers for simplicity)
        s: stride (assuming the same for all layers for simplicity)
        num_layers: number of convolutional layers
        '''
        output_size = n
        for _ in range(num_layers):
            output_size = int((output_size - f + 2 * p) / s) + 1
        return output_size
    
    def _dense_layer(self, n_in, n_out):
        return nn.Sequential(
            nn.Linear(n_in, n_out),
            nn.BatchNorm1d(n_out),
            nn.ReLU(),
            nn.Dropout(p=self.dense_dropout_rate)
        )

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        # print('=== before starting layer', x.size())
        x = self.starting_layers(x)
        # print('=== end of starting layer', x.size())
        x = x.view(x.size(0), -1)
        # print('=== Flattened', x.size())
        
        current_num_filters1 = self.cnn_output_size_multilayer(self.in_channel, 12, num_layers=2)
        # print(current_num_filters1, current_num_filters2)

        for layer in self.dense_layers:
            x = layer(x)
        # print('=== After Dense layer', x.size())

        out = self.out_layer(x)
        # print('After out layer', out.size())
        return out

torch.cuda.empty_cache()

epoch = 300
batch_size = 32
lr = 0.0001

model = Model(in_channel = 869, 
              first_h_layer = 469, 
              out_channel=1, 
              num_dense_layers=4,
              batch_size=batch_size)

model = model.float()
model = model.to(device)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, drop_last=True)
# criterion = nn.MSELoss()
criterion = masked_MSE
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)

# Training

In [9]:
torch.cuda.empty_cache()
import gc; gc.collect()
# ic.enable()
ic.disable()

train_epoch_loss = []
test_epoch_loss = []

for e in tqdm(range(1, epoch+1)):
    model.train()
    train_batch_loss = []
    test_batch_loss = []
    # print('epoch:', e)
    for x, y in train_loader:
        # print('batch x size:',x.size() )
        x_batch = torch.unsqueeze(x, 1).to(device)
        y_batch = y.to(device)
        x_batch = x_batch.float()
        y_batch = y_batch.float()
        # y_batch = y_batch.view(-1)

        # y_batch = one_hot_torch(y).to(device)
        # print('batch y size before flatten:',y_batch.size())
        # y_batch = y_batch.flatten()
        # print('batch y size after flatten:',y_batch.size())
        # print(x_batch.size())
        # print(x_batch.size())
# For example, if you have a convolutional layer with 64 output channels, 3 input channels, and a kernel size of 3x3, the weight parameters would have a dimension of (64, 3, 3, 3)
        # print(x_batch.size())
        pred = model(x_batch.float())
        # print(x_batch)
        # print(pred)
        # pred = pred.unsqueeze(0)
        # ic(pred)
        # ic(y_batch)
        ic(pred.size())
        loss_train = criterion(y_batch, pred)
        ic(loss_train)
        train_batch_loss.append(loss_train.detach())
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        # print(f'Batch - GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB')

    train_epoch_loss.append(torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy())
    with torch.no_grad():
        # print('>> test')
        # for x, y in test_loader:
            # print('x size:',x.size() )
        x_batch = torch.unsqueeze(x, 1).to(device)
        # print('batch x size:',x_batch.size() )
        y_batch = y.to(device)
        # print(x_batch.size())
        # y_batch = torch.Tensor.float(y).to(device)
        # x_batch = x_batch.permute(0, 3, 1, 2).to(device)
        pred = model(x_batch.float())
        # pred = pred.unsqueeze(0)

        loss_test = criterion(y_batch, pred)
        test_batch_loss.append(loss_test )
    test_epoch_loss.append(torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy())
        
    print(f'Epoch {e}')
    print(f"Training loss: {torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy()}")
    print(f"Validation loss: {torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy()}") 
    # scheduler.step(torch.mean(torch.stack(test_batch_loss)))
    # print(train_batch_loss)
    # print(test_batch_loss)
    # print(f"Training loss: {np.mean(train_batch_loss)}")
    # print(f"Validation loss: {np.mean(test_batch_loss)}")
print('==='*10)
# torch.save(model.state_dict(), '/mnt/storageG1/lwang/Projects/tb_dr_MIC/saved_models/rif_model.pt')
# fig, ax = plt.subplots()
# x = np.arange(1, epoch+1, 1)
# ax.plot(x, train_epoch_loss,label='Training')
# ax.plot(x, test_epoch_loss,label='Validation')
# ax.legend()
# ax.set_xlabel("Number of Epoch")
# ax.set_ylabel("Loss")
# ax.set_xticks(np.arange(0, epoch+1, 10))
# ax.set_title(f'Loss: Learning_rate:{lr}, cnn_dr:{cnn_dr}, cnn_dr:{fc_dr}')
# # ax_2 = ax.twinx()
# # ax_2.plot(history["lr"], "k--", lw=1)
# # ax_2.set_yscale("log")
# # ax.set_ylim(ax.get_ylim()[0], history["training_losses"][0])
# ax.grid(axis="x")
# fig.tight_layout()
# fig.show()
# fig.savefig(f'./graphs1/loss_lr_{lr}_cnn_dr_{cnn_dr}_fc_dr_{fc_dr}.png')
# print(f'./graphs1/loss_lr_{lr}_cnn_dr_{cnn_dr}_fc_dr_{fc_dr}.png')

  0%|          | 1/300 [00:30<2:33:49, 30.87s/it]

Epoch 1
Training loss: 49.66083908081055
Validation loss: 55.50987452682339


  1%|          | 2/300 [01:00<2:28:38, 29.93s/it]

Epoch 2
Training loss: 33.56352996826172
Validation loss: 29.15038745023667


  1%|          | 3/300 [01:29<2:26:46, 29.65s/it]

...

Epoch 298
Training loss: 7.509354591369629
Validation loss: 8.130785662576493


100%|█████████▉| 299/300 [2:25:46<00:29, 29.22s/it]

Epoch 299
Training loss: 7.443146705627441
Validation loss: 8.340312315299208


100%|██████████| 300/300 [2:26:16<00:00, 29.25s/it]

Epoch 300
Training loss: 7.466678142547607
Validation loss: 7.859086402183354


In [72]:
def save_to_file(file_path, appendix, epoch, lr, dr, train_loss, test_loss):
    train_loss = [float(arr) for arr in train_loss]
    test_loss = [float(arr) for arr in test_loss]

    with open(file_path, "a") as f:
        f.write(f">> {appendix}, Epoch: {epoch}, LR: {lr}, DR: {dr}\n")
        f.write(f"--- Train Loss: {train_loss}\n")
        f.write(f"--- Test Loss: {test_loss}\n")
        
def training(appendix:str, epoch:int, dropout_rate:float, lr:float, batch_size:int=16, train_dataset=train_dataset, val_dataset=val_dataset, verbose:bool=False, graphics:bool=False):
    print(f"====lr: {lr}, dropout rate: {dropout_rate}")
    torch.cuda.empty_cache()
    import gc; gc.collect()
    # ic.enable()
    ic.disable()
    
    model = Model(in_channel = 869, 
              first_h_layer = 469, 
              out_channel=1, 
              num_dense_layers=4,
              batch_size=batch_size,
              dropout_rate=dropout_rate)
    
    model = model.float()
    model = model.to(device)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, drop_last=True)
    # criterion = nn.MSELoss()
    criterion = masked_MSE
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)

    train_epoch_loss = []
    test_epoch_loss = []

    for e in tqdm(range(1, epoch+1)):
        model.train()
        train_batch_loss = []
        test_batch_loss = []
        
        for x, y in train_loader:
            x_batch = torch.unsqueeze(x, 1).to(device)
            print(x_batch.size())
            # x_batch = x.permute(1, 0, 2)
            # print(x_batch.size())
            y_batch = y.to(device)
            x_batch = x_batch.float()
            y_batch = y_batch.float()

            pred = model(x_batch.float())

            ic(pred.size())
            loss_train = criterion(y_batch, pred)
            ic(loss_train)
            train_batch_loss.append(loss_train.detach())
            
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

            # print(f'Batch - GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB')

        train_epoch_loss.append(torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy())
        with torch.no_grad():
            # print('test')
            for x, y in test_loader:
                x_batch = x.to(device)
                y_batch = y.to(device)
                pred = model(x_batch.float())
                loss_test = criterion(y_batch, pred)
                test_batch_loss.append(loss_test )
            test_epoch_loss.append(torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy())
        if verbose:
            print(f'Epoch {e}')
            print(f"Training loss: {torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy()}")
            print(f"Validation loss: {torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy()}") 
            print('==='*10)
        
    save_to_file('trials_rif.txt', appendix ,epoch, lr, dropout_rate, train_epoch_loss, test_epoch_loss)

    if graphics:
        fig, ax = plt.subplots()
        x = np.arange(1, epoch+1, 1)
        ax.plot(x, train_epoch_loss,label='Training')
        ax.plot(x, test_epoch_loss,label='Validation')
        ax.legend()
        ax.set_xlabel("Number of Epoch")
        ax.set_ylabel("Loss")
        ax.set_xticks(np.arange(0, epoch+1, 10))
        ax.set_title(f'Loss: Learning_rate:{lr},dr:{dropout_rate}')
        # ax_2 = ax.twinx()
        # ax_2.plot(history["lr"], "k--", lw=1)
        # ax_2.set_yscale("log")
        # ax.set_ylim(ax.get_ylim()[0], history["training_losses"][0])
        ax.grid(axis="x")
        fig.tight_layout()
        fig.savefig(f'/mnt/storageG1/lwang/Projects/tb_dr_MIC/graph_rif/{appendix}_loss_lr_{lr}_dr_{dropout_rate}.png')
        fig.show()
    return torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy(), torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy()
# training(epoch=50, dropout_rate=0.2, lr=0.001, batch_size=128, train_dataset=train_dataset, val_dataset=val_dataset, graphics=True)

In [74]:
lr_values = [1e-5, 1e-3, 1e-1,]
# dropout_rates = [0.2, 0.3, 0.4,0.5]
dropout_rates = [0]

# lr_values = [0.001]
# dropout_rates = [0.5]

results_test = []
results_train = []

for lr in lr_values:
    for dropout_rate in dropout_rates:
        result_test, result_train = training('5layers', epoch=100, dropout_rate=dropout_rate, lr=lr, batch_size=16, train_dataset=train_dataset, val_dataset=val_dataset, graphics=True)
        results_test.append((lr, dropout_rate, result_test))
        results_train.append((lr, dropout_rate, result_train))
# Sort the results based on the validation loss

# with open(f'results_test_lr{lr}_dr{dropout_rate}.txt', 'w') as file:
#     for lr, dropout_rate, result in results_test:
#         file.write(f'Learning Rate: {lr}, Dropout Rate: {dropout_rate}, Test Result: {result}\n')

# # Save the train results to a text file
# with open(f'results_train_lr{lr}_dr{dropout_rate}.txt', 'w') as file:
#     for lr, dropout_rate, result in results_train:
#         file.write(f'Learning Rate: {lr}, Dropout Rate: {dropout_rate}, Train Result: {result}\n')

results_test.sort(key=lambda x: x[2])

# Print the best lr and dropout rate
best_lr, best_dropout_rate, best_result = results_test[0]
print(f"Best lr: {best_lr}, Best dropout rate: {best_dropout_rate}, Best result: {best_result}")

====lr: 1e-05, dropout rate: 0


  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([16, 1, 869])


AttributeError: 'Model' object has no attribute 'cnn_output_size'